In [10]:
from surprise import Reader
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [16]:
data = Dataset.load_builtin('ml-100k')
data

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\User/.surprise_data/ml-100k


In [19]:
trainset,testset = train_test_split(data,test_size=.25,random_state=0)

algo = SVD(n_factors=50,random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
print('prediction type',type(predictions),'size:',len(predictions))
print('prediction 5개최초 결과물\n',predictions[:5])

prediction type <class 'list'> size: 25000
prediction 5개최초 결과물
 [Prediction(uid='120', iid='282', r_ui=4.0, est=3.6532744639165786, details={'was_impossible': False}), Prediction(uid='882', iid='291', r_ui=4.0, est=3.9097372780826998, details={'was_impossible': False}), Prediction(uid='535', iid='507', r_ui=5.0, est=3.8776919127707177, details={'was_impossible': False}), Prediction(uid='697', iid='244', r_ui=5.0, est=3.704888702614823, details={'was_impossible': False}), Prediction(uid='751', iid='385', r_ui=4.0, est=3.3572494565126663, details={'was_impossible': False})]


In [20]:
[(pred.uid,pred.iid,pred.est) for pred in predictions[:3]]

[('120', '282', 3.6532744639165786),
 ('882', '291', 3.9097372780826998),
 ('535', '507', 3.8776919127707177)]

In [21]:
# predict() 메서드 이용해서 사용자의 아이템에 대한 추천 평점예측
#사용자 아이디,아이템 아이디 문자열로 입력해야 함
uid = str(196)
iid =str(302)
pred = algo.predict(uid,iid)
pred

Prediction(uid='196', iid='302', r_ui=None, est=4.37904004664696, details={'was_impossible': False})

In [22]:
# 추천예측평점과 실제 평점과의 차이를 평가
accuracy.rmse(predictions)

RMSE: 0.9457


0.9456994032241722

In [ ]:
# ,다른 데이터셋 불러올 때에도 사용자아이디(uid), 아이템아이디(iid),평점(est) 순으로 돼 있어야 함

In [23]:
import pandas as pd

ratings = pd.read_csv('./dataset/ml-latest-small/ratings.csv')
# 헤더 삭제
ratings.to_csv('./dataset/ml-latest-small/ratings_noh.csv',index=False,header=False)
#파싱정보 알려주기 위해 사용, 평점의 단위는 0.5 최대 평점은 5
reader= Reader(line_format='user item rating timestamp',sep=',',rating_scale =(0.5,5))
data=Dataset.load_from_file('./dataset/ml-latest-small/ratings_noh.csv',reader=reader)

In [30]:
print(ratings.head())
ratings.info()

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [25]:
trainset, testset = train_test_split(data,test_size=.25,random_state=0)
#수행시마다 동일한 결과물 출력위해 random_state=0 부여
algo = SVD(n_factors = 50,random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [28]:
import pandas as pd
from surprise import Reader,Dataset

ratings = pd.read_csv('./dataset/ml-latest-small/ratings.csv')
reader=Reader(rating_scale=(0.5,5))

data= Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
trainset,testset = train_test_split(data,test_size=.25,random_state=0)

algo =SVD(n_factors=50,random_state=0)
algo.fit( trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [44]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('./dataset/ml-latest-small/ratings.csv')
reader=Reader(rating_scale=(0.5,5))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

algo =SVD(random_state=0)
cross_validate(algo,data,measures=['RMSE','MAE'],cv=5,verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8721  0.8760  0.8813  0.8611  0.8787  0.8738  0.0071  
MAE (testset)     0.6706  0.6724  0.6759  0.6632  0.6742  0.6713  0.0044  
Fit time          7.48    7.20    9.51    8.01    7.70    7.98    0.81    
Test time         0.21    0.23    0.21    0.22    0.24    0.22    0.01    


{'test_rmse': array([0.87211918, 0.87598203, 0.8812799 , 0.86105294, 0.8787288 ]),
 'test_mae': array([0.67058793, 0.6724283 , 0.67590378, 0.66317034, 0.67419109]),
 'fit_time': (7.483924150466919,
  7.19727087020874,
  9.514805316925049,
  8.012245893478394,
  7.7018210887908936),
 'test_time': (0.2104020118713379,
  0.23436975479125977,
  0.2050018310546875,
  0.22067904472351074,
  0.2443406581878662)}

In [48]:
from surprise.model_selection import GridSearchCV


param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])


0.8774607626618941
{'n_epochs': 20, 'n_factors': 50}


### surprise 를 이용한 개인화 영화 추천 시스템 구축

In [49]:
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
algo =SVD(n_factors=50,random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

In [53]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='./dataset/ml-latest-small/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [58]:
algo =SVD(n_factors=50,n_epochs=20,random_state=0)
algo.fit(trainset)
movies = pd.read_csv('./dataset/ml-latest-small/movies.csv')
#userId= 9의 moiveId  데이터를 추출해 movieId=42 데이터가 있는지 확인
moviesId = ratings[ratings['userId']==9]['movieId']
if moviesId[moviesId==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42 평점 없음')
print(movies[movies['movieId']==42])


사용자 아이디 9는 영화 아이디 42 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [59]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [80]:
movies[movies['movieId']==42]

,movieId,title,genres
38,42,Dead Presidents (1995),Action|Crime|Drama


In [104]:
uid = str(9)
iid = str(42)
pred = algo.predict(uid,iid,verbose=True)
pred


user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


Prediction(uid='9', iid='42', r_ui=None, est=3.130146490888994, details={'was_impossible': False})

In [113]:
def get_unseen_surprise(ratings,movies,userId):
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    total_movies = movies['movieId'].tolist()
    unseen_movies =[ movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수',len(seen_movies),'추천 영화 수',len(unseen_movies),'전체 영화수',len(total_movies))
    return unseen_movies
unseen_movies= get_unseen_surprise(ratings,movies,9)

평점 매긴 영화 수 46 추천 영화 수 9696 전체 영화수 9742


In [62]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [95]:
#최대 평점을 가진 행찾기 
ratings[ratings['userId']==610]['rating'].idxmax()

99534

In [103]:
ratings[ratings['userId']==610].idxmax()
# ratings[ratings['userId']==ratings['userId'].max()]

userId        99534
movieId      100835
rating        99534
timestamp     99775
dtype: int64

In [ ]:
movies[moviesId==42]